# Librerías

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np

# Datos

In [2]:
df = pd.read_csv("../Datos/RMCAB/Clean/Sec_Cruzada_RMCAB.csv")
df["Fecha_Hora"] = pd.to_datetime(df["Fecha_Hora"])

In [3]:
df.head(3)

,Fecha_Hora,Estación,BBP,BC ug/m3,CO,Dir Viento,Dir Viento 10M,HR,HR_2m,Humedad Inter,...,SO2,SO2 Envea,Temp_4m,Tempe Inter,Temperatura,Temperatura 20M,Temperatura 8M,UV-BC,Vel Viento,Vel Viento 10M
0,2000-01-01,Carvajal - Sevillana,NaN,NaN,0.7,306.0,NaN,NaN,NaN,NaN,...,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN
1,2000-01-01,Guaymaral,NaN,NaN,NaN,NaN,194.0,94.01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2000-01-01,MinAmbiente,NaN,NaN,NaN,293.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,NaN


# Identificación de estaciones

In [4]:
estaciones = df["Estación"].unique().tolist()
info_estaciones = pd.DataFrame(columns=["Estación", "Min_Fecha_Hora"])

# Iterate over each station and find the minimum date and time
for estacion in estaciones:
    min_date = df[df["Estación"] == estacion]["Fecha_Hora"].min()
    new_row = pd.DataFrame({"Estación": [estacion], "Min_Fecha_Hora": [min_date]})
    info_estaciones = pd.concat([info_estaciones, new_row], ignore_index=True)

In [5]:
estaciones_filtradas = info_estaciones.head(7)["Estación"].tolist()
estaciones_filtradas

['Carvajal - Sevillana',
 'Guaymaral',
 'MinAmbiente',
 'Puente Aranda',
 'Suba',
 'Usaquen',
 'Las Ferias']

# Info variables por estación

In [6]:
df = df[df["Estación"].isin(estaciones_filtradas)]
variables = df.columns[2:].tolist()

In [7]:
try:
    df.set_index("Fecha_Hora", inplace=True)
except:
    pass


fechas_variables = []

for estacion in estaciones_filtradas:
    for var in variables:
        var_df = pd.DataFrame((df.loc[df["Estación"]==estacion])[var])
        var_df = var_df.dropna()
        min_date = var_df.index.min()
        fechas_variables.append({
            "Estación": estacion,
            "Variable": var,
            "Fecha Inicio": min_date
        })

# Convertir la lista de diccionarios en un DataFrame
fechas_variables_df = pd.DataFrame(fechas_variables)
fechas_variables_df = fechas_variables_df.pivot_table(index="Estación", columns="Variable", values="Fecha Inicio")

In [8]:
#fechas_variables_df.to_excel("./Fechas_Inicio_Variables.xlsx")
#correlations_by_estacion = df.groupby("Estación").apply(lambda x: x.drop(columns=["Estación"]).corr())
#correlations_by_estacion

# Análisis exploratorio

## Disponibilidad estatica

In [22]:

df1 = pd.DataFrame(columns=["Estación", "Variable", "Disp_2000", "Disp_inicial", "Fecha_inicial"])

for estacion in estaciones_filtradas:

    e = df[df["Estación"] == estacion]

    disp_2000 = []
    disp_principio = []
    disp_var = []
    fechas_iniciales = []
    for var in variables:

        df_disp = e[var]
        disponibilidad_2000 = df_disp.isna().sum() / len(df_disp)
        fecha_inicial = df_disp.dropna().index.min()
        disp_prin = df_disp.loc[df_disp.index >= fecha_inicial].isna().sum() / len(df_disp.loc[df_disp.index >= fecha_inicial])

        disp_2000.append(1-disponibilidad_2000)
        disp_var.append(var)
        disp_principio.append(1-disp_prin)

        fechas_iniciales.append(fecha_inicial)

    disp_df = pd.DataFrame({"Variable": disp_var, "Disp_2000": disp_2000,"Disp_inicial":disp_principio, "Fecha_inicial":fechas_iniciales}).round(2)
    disp_df["Estación"] = estacion
    disp_df = disp_df.sort_values(by="Disp_inicial", ascending=False)
    disp_df = disp_df.dropna()
    df1 = pd.concat([df1, disp_df], ignore_index=True)

## Disponibilidad a lo largo del tiempo

In [10]:
# Crear un dataframe vacío para almacenar resultados
dfs2 = pd.DataFrame(columns=["Estación", "Variable", "Año", "Disponibilidad"])

# Iterar por cada estación
for estacion in estaciones_filtradas:
    
    # Filtrar datos por estación
    e = df[df["Estación"] == estacion]
    
    # Iterar por cada variable
    for var in variables:
        
        # Extraer los datos de la variable en la estación correspondiente
        df_disp = e[var]
        
        # Iterar por cada año presente en los datos
        for year in df_disp.index.year.unique():
            
            # Filtrar los datos por año
            df_year = df_disp[df_disp.index.year == year]
            
            # Calcular el porcentaje de datos no nulos
            disponibilidad_anual = df_year.notna().sum() / len(df_year)
            
            # Crear un nuevo registro con los resultados
            disp_df = pd.DataFrame({
                "Estación": [estacion],
                "Variable": [var],
                "Año": [year],
                "Disponibilidad": [disponibilidad_anual]
            })
            
            # Concatenar el resultado al dataframe general
            dfs2 = pd.concat([dfs2, disp_df], ignore_index=True)

# Redondear los resultados a 2 decimales
dfs2 = dfs2.round(2)
dfs2["Año"] = dfs2["Año"].astype(int)
dfs2 = dfs2.loc[dfs2["Disponibilidad"] > 0]
dfs2

,Estación,Variable,Año,Disponibilidad
50,Carvajal - Sevillana,CO,2000,0.92
51,Carvajal - Sevillana,CO,2001,0.56
52,Carvajal - Sevillana,CO,2002,0.27
53,Carvajal - Sevillana,CO,2003,0.90
54,Carvajal - Sevillana,CO,2004,0.12
...,...,...,...,...
4758,Las Ferias,Vel Viento,2020,0.93
4759,Las Ferias,Vel Viento,2021,1.00
4760,Las Ferias,Vel Viento,2022,1.00
4761,Las Ferias,Vel Viento,2023,0.97


## Visualizaciones 

### Serie de tiempo

In [11]:
#for estacion in estaciones_filtradas:
#    
#    vis_df = dfs[dfs["Estación"] == estacion]
#    for var in vis_df["Variable"].tolist():
#
#        plt.figure(figsize=(10, 6))
#        plt.plot(df.loc[df["Estación"] == estacion, var])
#        plt.title(f"{estacion} - {var}")
#        plt.show()

### Disponibilidad

In [12]:

#dfs["Año"] = pd.to_datetime(dfs["Año"])
#dfs = dfs.loc[dfs["Año"] != "2024"]
#for estacion in estaciones_filtradas:
#    vis_df = dfs.loc[dfs["Estación"] == estacion]
#    for var in vis_df["Variable"].unique():
#        var_df = vis_df[vis_df["Variable"] == var]
#        plt.figure(figsize=(10, 6))
#        plt.bar(var_df["Año"], var_df["Disponibilidad"])
#        plt.title(f"Disponibilidad por año en {estacion} - {var}")
#        plt.xlabel("Año")
#        plt.ylabel("Disponibilidad")
#        plt.grid(True)
#        plt.show()

In [ ]:
# Filtrar para excluir el año 2024
dfs2 = dfs2.loc[dfs2["Año"] != "2024"]
dfs2["Año"] = dfs2["Año"].astype(int)

# Definir el número de columnas
cols = 5

# Iterar por cada estación
for estacion in estaciones_filtradas:
    # Filtrar los datos por estación
    vis_df = dfs2.loc[dfs2["Estación"] == estacion]
    
    # Obtener las variables únicas para esa estación
    variables = vis_df["Variable"].unique()
    
    # Calcular el número de filas necesarias
    rows = int(np.ceil(len(variables) / cols))
    
    # Crear una figura con subplots, distribuidos en 'rows' filas y 'cols' columnas
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(30, 18))
    
    # Aplanar el array de axes para facilitar la iteración
    axes = axes.flatten()
    
    # Iterar por cada variable y graficar
    for i, var in enumerate(variables):
        # Filtrar los datos por variable
        var_df = vis_df[vis_df["Variable"] == var]
        
        # Graficar barras para la variable en el subplot correspondiente
        axes[i].bar(var_df["Año"].astype(int), var_df["Disponibilidad"])
        
        # Añadir título y etiquetas
        axes[i].set_title(f"{var}")
        axes[i].set_xlabel("Año")
        axes[i].set_ylabel("Disponibilidad")
        axes[i].grid(True)
    
    # Si hay subplots vacíos (cuando el número de variables no es múltiplo de 5)
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])  # Eliminar los subplots sobrantes
    
    # Ajustar el layout para evitar solapamientos
    plt.suptitle(f"Disponibilidad por año en {estacion}", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Ajustar para no cortar el título
    
    # Mostrar la figura con todos los subplots
    plt.show()

## Imputación

In [26]:
df1 = df1.loc[df1["Estación"] == "Guaymaral"].sort_values(by="Disp_inicial", ascending=False)
df1 = df1.loc[df1["Disp_inicial"] > 0.7]

var_to_imput = df1["Variable"].tolist()

In [39]:
df1

,Estación,Variable,Disp_2000,Disp_inicial,Fecha_inicial
11,Guaymaral,Temperatura,0.69,0.99,2008-02-29 00:00:00
12,Guaymaral,CO,0.13,0.97,2021-06-01 00:00:00
13,Guaymaral,Precipitacion,0.97,0.97,2000-01-01 00:00:00
14,Guaymaral,OZONO,0.63,0.92,2008-06-03 00:00:00
15,Guaymaral,PM10,0.89,0.92,2001-05-02 09:00:00
16,Guaymaral,HR,0.90,0.90,2000-01-01 00:00:00
17,Guaymaral,Presion Baro,0.85,0.85,2000-01-01 00:00:00
18,Guaymaral,NO,0.54,0.78,2008-05-26 00:00:00
19,Guaymaral,NO2,0.54,0.78,2008-05-26 00:00:00


### Ejemplo variable estación guaymaral

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor


In [286]:
e = df.loc[df["Estación"] == "Guaymaral"]
var = "Presion Baro"
min_date = df1.loc[df1["Variable"] == var, "Fecha_inicial"].values[0]

ex_var = e[var].loc[e[var].index >= min_date]
ex_var = ex_var.to_frame()
first_nan_date = ex_var[ex_var.isna().any(axis=1)].index[0]

model_df = ex_var.loc[ex_var.index <= first_nan_date]

lags = 3
for lag in range(1, lags + 1):
    model_df[f'lag_{lag}'] = model_df[var].shift(lag)
model_df = model_df.dropna()

X = model_df.drop(columns=[var])
y = model_df[var]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

next_block = ex_var.loc[ex_var.index >= first_nan_date]
second_nan_date = next_block[next_block.isna().any(axis=1)].index[0]

nan_block_to_replace = ex_var.loc[(ex_var.index >= first_nan_date) & (ex_var.index <= second_nan_date)]

for lag in range(1, lags + 1):
    nan_block_to_replace[f'lag_{lag}'] = ex_var[var].shift(lag).loc[nan_block_to_replace.index]
#nan_block_to_replace = nan_block_to_replace.dropna()
nan_block_to_replace
nan_block_to_replace["Presion Baro"] = model.predict(nan_block_to_replace.drop(columns=["Presion Baro"])).round(0)
ex_var.loc[nan_block_to_replace.index, "Presion Baro"] = nan_block_to_replace["Presion Baro"]

first_nan_date = ex_var[ex_var.isna().any(axis=1)].index[0]

model_df = ex_var.loc[ex_var.index <= first_nan_date]

lags = 3
for lag in range(1, lags + 1):
    model_df[f'lag_{lag}'] = model_df[var].shift(lag)
model_df = model_df.dropna()

X = model_df.drop(columns=[var])
y = model_df[var]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

next_block = ex_var.loc[ex_var.index >= first_nan_date]
second_nan_date = next_block[next_block.isna().any(axis=1)].index[0]

nan_block_to_replace = ex_var.loc[(ex_var.index >= first_nan_date) & (ex_var.index <= second_nan_date)]

for lag in range(1, lags + 1):
    nan_block_to_replace[f'lag_{lag}'] = ex_var[var].shift(lag).loc[nan_block_to_replace.index]
#nan_block_to_replace = nan_block_to_replace.dropna()
nan_block_to_replace
nan_block_to_replace["Presion Baro"] = model.predict(nan_block_to_replace.drop(columns=["Presion Baro"])).round(0)
ex_var.loc[nan_block_to_replace.index, "Presion Baro"] = nan_block_to_replace["Presion Baro"]

Mean Absolute Percentage Error (MAPE): 0.33%
Mean Absolute Percentage Error (MAPE): 0.32%


In [ ]:
jeje = ex_var.head(1000).tail(500)
jeje

In [297]:
ex_var = ex_var.head(30000)
ex_var.isna().sum()

Presion Baro    474
dtype: int64

In [ ]:
# Filtrar la estación de interés
e = df.loc[df["Estación"] == "Guaymaral"]
var = "Presion Baro"
min_date = df1.loc[df1["Variable"] == var, "Fecha_inicial"].values[0]

# Seleccionar la serie temporal de interés desde la fecha inicial
ex_var = e[var].loc[e[var].index >= min_date]
ex_var = ex_var.to_frame()

# Inicializar el número de lags
lags = 3

ex_var = ex_var.head(30000)
ex_var.isna().sum()

# Bucle para rellenar valores nulos y reentrenar el modelo
while ex_var[var].isna().any():
    
    # Encontrar el primer índice de valor nulo
    first_nan_date = ex_var[ex_var.isna().any(axis=1)].index[0]

    # Crear conjunto de datos hasta el primer valor nulo
    model_df = ex_var.loc[ex_var.index < first_nan_date]

    # Generar las variables con lags
    for lag in range(1, lags + 1):
        model_df[f'lag_{lag}'] = model_df[var].shift(lag)
    model_df = model_df.dropna()

    # Dividir las características (X) y la variable objetivo (y)
    X = model_df.drop(columns=[var])
    y = model_df[var]

    # Separar en entrenamiento y prueba (opcional para evaluación)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Entrenar el modelo
    model = XGBRegressor()
    model.fit(X_train, y_train)

    # Predecir el siguiente bloque de valores nulos
    next_block = ex_var.loc[ex_var.index >= first_nan_date]
    
    # Identificar el siguiente valor no nulo después del primer valor nulo
    try:
        second_nan_date = next_block[next_block.notna().any(axis=1)].index[0]
    except IndexError:
        # Si no hay un segundo bloque no nulo (es decir, el bloque nulo llega hasta el final), utilizar el final del DataFrame
        second_nan_date = ex_var.index[-1]

    # Seleccionar el bloque de valores nulos para predecir
    nan_block_to_replace = ex_var.loc[(ex_var.index >= first_nan_date) & (ex_var.index < second_nan_date)]
    
    # Crear las características (lags) para las predicciones
    for lag in range(1, lags + 1):
        nan_block_to_replace[f'lag_{lag}'] = ex_var[var].shift(lag).loc[nan_block_to_replace.index]
    
    # Eliminar filas sin suficientes lags (opcional)
    nan_block_to_replace = nan_block_to_replace.dropna()

    # Realizar la predicción del bloque de nulos
    X_pred = nan_block_to_replace.drop(columns=[var])
    y_pred = model.predict(X_pred).round(0)  # Redondear si es necesario
    
    # Rellenar el DataFrame original con las predicciones
    ex_var.loc[nan_block_to_replace.index, var] = y_pred

    # Reentrenar el modelo con los nuevos datos hasta la última fecha con valores reemplazados
    model_df = ex_var.loc[ex_var.index <= second_nan_date]
    print(ex_var.isna().sum())  # Imprimir la cantidad de valores nulos restantes en cada iteración

# Fin del bucle: cuando ya no haya más valores nulos
print("Todos los valores nulos han sido reemplazados.")

In [ ]:
ex_var.isna().sum()